In [1]:
from random import *
import hashlib

def puissance (a,e,n):
  p = 1
  while e>0:
    if (e % 2 != 0):
      p=(p*a)%n
    a=(a*a)%n
    e=e//2
  return p

In [2]:
# Test puissance
def test_puissance():
    assert(puissance(4,2,5)==1)
test_puissance()

In [3]:
def test_premier_Fermat(n): # test de Fermat
   if ( (puissance(2,n-1,n)==1) and
      (puissance(3,n-1,n)==1) and
      (puissance(5,n-1,n)==1) and
      (puissance(7,n-1,n)==1) and
      (puissance(11,n-1,n)==1) and
      (puissance(13,n-1,n)==1) ):
      return True; # probablement premier (garantie si n<2^15)
   return False

In [4]:
# Test premier de Fermat
def test_Fermat():
    assert(test_premier_Fermat(22)==False)
    assert(test_premier_Fermat(647)==True)
test_Fermat()

In [5]:
def temoin_Miller(n, a):
    s = 0
    d = n-1
    while d % 2 == 0:
       d = d//2
    x = puissance(a, d, n)
    if(x == 1 or x == n-1):
        return False
    for i in range(s-1):
        x = puissance(x, 2, n)
        if(x == n-1):
            return False
    return True

In [6]:
# Test témoin de Miller 
def test_temoin_Miller():
    assert(temoin_Miller(3,9)==True)
    assert(temoin_Miller(4,9)==False)
test_temoin_Miller()

In [7]:
# Le GMP recommande k=25 pour avoir une proba d'erreur < 2**-50
def test_premier_Miller_Rabin(n, k):
    if n == 2 or n == 3:
       return True
    if n % 2 == 0:
       return False
    for i in range(k):
        a = randint(2, n-2)
        if(temoin_Miller(n, a)):
            return False
    return True

In [8]:
# Test de primalité de Miller Rabin
def test_premier_Miller():
    assert(test_premier_Miller_Rabin(3,1)==True)
    assert(test_premier_Miller_Rabin(42,5)==False)
test_premier_Miller()

In [9]:
def pgcd(u, v):
  t = 0
  while (v):
    t = u
    u = v
    v = t % v
  if(u<0):
    return -u
  else:
    return u

In [10]:
# Test PGCD
def test_pgcd():
    assert(pgcd(5,4)==1)
    assert(pgcd(222,448)==2)
test_pgcd()

In [11]:
def bezout(a, b):

  # On sauvegarde les valeurs de a et b.
   a0 = a
   b0 = b

  # On laisse invariant p*a0 + q*b0 = a et  r*a0 + s*b0 = b.
   p = 1
   q = 0
   r = 0
   s = 1
   c = 1
   quotient = 1
   nouveau_r = 1
   nouveau_s = 1

  # La boucle principale.
   while (b != 0):
    c = a % b
    quotient = a//b
    a = b
    b = c
    nouveau_r = p - (quotient * r)
    nouveau_s = q - (quotient * s)
    p = r
    q = s
    r = nouveau_r
    s = nouveau_s
   if p < 0:
     p = p+b0
   return p  # on n'a besoin que de p #si p est négatif retourne p+phi

In [12]:
# Test théorème de Bezout
def test_Bezout():
    assert(bezout(50,47)==16)
    assert(bezout(15,5)==0)
    assert(bezout(445,457)==38)
test_Bezout()

In [53]:
def generateurClé(c,b):
  p=randint(c,b)
  q=randint(c,b)
  e=randint(c,b)
  d=randint(c,b)

  while test_premier_Miller_Rabin(p,25)==False:
    p = randint(c,b)
  while test_premier_Miller_Rabin(q,25)==False:
    q = randint(c,b)

  n=p*q
  
  phi = (p - 1)*(q - 1)
 
  while 1>=e>=phi or pgcd(e,phi)!=1 :
      e = randint(c,b)
  d= int(bezout(int(e),int(phi)))
  return e,d,n 

(41, 1541, 2449)

In [54]:
def chiffrer(message,clé,n):
    return puissance(message,clé,n)

1927

In [56]:
def test_chiffrer():
    assert(chiffrer(1234,41,2449)==1927)
test_chiffrer()

In [55]:
def dechiffrer(messageChiffre,clé,n):
    return puissance(messageChiffre,clé,n)

1234

In [57]:
def test_dechiffrer():
    assert(dechiffrer(1927,1541,2449)==1234)
test_dechiffrer()

In [60]:
def str_to_int(m):
    s=0
    b=1
    for i in range(len(m)):
        s=s+ord(m[i])*b
        b=b*256
    return s

107

In [59]:
def test_str_to_int():
    assert(str_to_int('a')==97)
    assert(str_to_int('k')==107)
test_str_to_int()

In [68]:
def int_to_str(c):
    s =""
    q,r= divmod(c,256)
    s=s+str(chr(r))
    while q != 0:
        q,r = divmod(q,256)
        s =s +str(chr(r))
    return s 


In [70]:
def test_int_to_str():
    assert(int_to_str(97)=='a')
    assert(int_to_str(107)=='k')
test_int_to_str()

In [71]:
def calculEmpreinte(message): 
    m=hashlib.sha256(repr(message).encode())
    return m.hexdigest()

'03ac674216f3e15c761ee1a5e255f067953623c8b388b4459e13f978d7c846f4'

In [19]:
def decoupe(variable,taille,liste,strVariable):
    decoupeIndex = 0
    for i in range(1,len(variable)):
        if(variable[i]!='0' and i%taille==0):
            liste.append(int(variable[decoupeIndex:i]))
            strVariable+=str(int(variable[decoupeIndex:i]))   
            decoupeIndex=i
        if(i==len(variable)-1):
            liste.append(int(variable[decoupeIndex:len(variable)]))
            strVariable+=str(int(variable[decoupeIndex:len(variable)]))

In [75]:
def test_decoupe():
    l=[]
    strL = ''
    liste=[]
    strListe = '123456'
    liste.append(12)
    liste.append(34)
    liste.append(56)
    decoupe(strListe,2,l,strL)
    assert(l== liste)
    
test_decoupe()

In [78]:
def chiffrerEnsemble(liste, cle):
       for i in range(len(liste)):
            liste[i]=chiffrer(liste[i],cle[0],cle[1])

In [86]:
# Test chiffrer ensemble 
def test_chiffrerEnsemble():
    liste = []
    liste.append(12)
    liste.append(34)
    liste.append(56)
    gC=generateurClé(10**1,10**2)
    clé= [gC[0],gC[2]]
    chiffrerEnsemble(liste,clé)
    l =[]
    l.append(chiffrer(12,clé[0],clé[1]))
    assert(l==liste)

In [85]:
def envoieAuthentifie(message,clePrivee,clePublique):
    #Verifie si le message est un string, si oui le convertit en int
    strEmpreinte=""
    strEmpreinteC=""
    messageEmpreinte=[]
    messageParti=[]
    strMessage=""
    if(isinstance(message,str)):
        message=str_to_int(message)
    
    #Calcule de l'empreinte du message
    empreinte =str(str_to_int(calculEmpreinte(message)))
    tailleMessage=int(len(str(message))/(len(str(message))/2))
    tailleEmpreinte=int(len(empreinte)/(len(empreinte)/2))
    
    #Découpe de l'empreinte en 10 pour pouvoir mieux la chiffrer   
    decoupe(empreinte,tailleEmpreinte,messageEmpreinte,strEmpreinte) 
    print("Empreinte: "+empreinte)
    
    #On decoupe le message en plusieurs morceaux
    decoupe(str(message),tailleMessage,messageParti,strMessage)
    
    #Chiffrage de l'empreinte avce la clé privée de l'envoyeur
    for i in range(0,len(messageEmpreinte)):
        messageEmpreinte[i]=chiffrer(messageEmpreinte[i],clePrivee[0],clePrivee[1])
        strEmpreinteC+=str(messageEmpreinte[i])
    print("Empreinte Chiffré: "+strEmpreinteC)
    
    #Concaténation du message et de l'empreinte chiffrée
    print("Message + empreinte: "+strMessage+strEmpreinteC)
    
    #On chiffre l'ensemble des empreintes et des messages
    chiffrerEnsemble(messageEmpreinte,clePublique)
    chiffrerEnsemble(messageParti,clePublique)   
        
    #Envoie du message finale chiffrée avec la clé publique du receveur
    return messageParti,messageEmpreinte



In [22]:
def verif_signature(messageParti,messageEmpreinte,clePrivee,clePublique):
    messageEmpreinteDecryptage = messageEmpreinte
    messagePartiDecryptage= messageParti
    empreinte1=""
    empreinteDecrypt=""
    message=""
    
    #Dechiffrage de l'empreinte
    for i in range(len(messageEmpreinteDecryptage)):
      messageEmpreinteDecryptage[i] = dechiffrer(messageEmpreinteDecryptage[i],clePrivee[0],clePrivee[1])
      #print("Empreinte bout déchiffré: "+str(messageListDecryptage[i]))
      empreinte1 += str(messageEmpreinteDecryptage[i])
    
    #Dechiffrage du message
    for i in range(len(messagePartiDecryptage)):
      messagePartiDecryptage[i]=dechiffrer(messagePartiDecryptage[i],clePrivee[0],clePrivee[1])
      message+=str(messagePartiDecryptage[i])
    
    print("Message déchiffré: "+message+empreinte1)
    print("Message : "+message)
    print("Empreinte du message chiffrée: "+empreinte1)
    
    
    #Déchiffrage de l'empreinte avec la clé publique
    for i in range(len(messageEmpreinteDecryptage)):
      messageEmpreinteDecryptage[i] = str(dechiffrer(messageEmpreinteDecryptage[i],clePublique[0],clePublique[1]))
      empreinteDecrypt+= str(messageEmpreinteDecryptage[i])
    
    #Vérification que l'empreinte déchiffrer correspond à celui du message
    if(str_to_int(calculEmpreinte(int(message))))==(int(empreinteDecrypt)):
       return int(message) #int_to_str() demander si 2 fonctions différente ou 1 seule
    else:
       return print("Le message a été modifier")
    

In [23]:
#Alice
gA=generateurClé(10**38,10**39)
cléPubliqueA= [gA[0],gA[2]]
cléPrivéeA= [gA[1],gA[2]]
print("Clé Publique A: "+str(cléPubliqueA))
print("Clé Privée A: "+str(cléPrivéeA))

Clé Publique A: [566104928636221625278900332184719948257, 494854217550269462419805614283075716723252814616104313065543939580087400475777]
Clé Privée A: [213944358951125613117305229772545862213787246314345379031912110533810681482533, 494854217550269462419805614283075716723252814616104313065543939580087400475777]


In [24]:
#CA
gCA=generateurClé(10**56,10**57) 
cléPubliqueCA= [gCA[0],gCA[2]]
cléPrivéeCA= [gCA[1],gCA[2]]
print("Clé Publique CA: "+str(cléPubliqueCA))
print("Clé Privée CA: "+str(cléPrivéeCA))

Clé Publique CA: [980618886526186726337594134931693363354417070010350000953, 227322133814453821886634017724633431611331132463817870381370968226483543290976484036272089971374842583495382356573]
Clé Privée CA: [162152109823466623786009504711900279193563110356730187294811123874855177071292324080008648861397180552137721428061, 227322133814453821886634017724633431611331132463817870381370968226483543290976484036272089971374842583495382356573]


In [25]:
#Envoie le message au CA
messageA=envoieAuthentifie(int(cléPubliqueA[0]),cléPrivéeA,cléPubliqueCA)

Empreinte: 5101009768712777106123675033215029161825677424745336421134716482427355462645015480394160321968172490879692925723593161952491183058471555446978353089832245
Empreinte Chiffré: 35535873742101440880763340823271203089966211091198966137260171524442151162278781342275837978377701535945210463117006314946002087778597956356322406004721063385255643955823012689436121743202004278342530996372038466518648856070189283699362850214786745725601099257048265312419961463631791281479755071649836295908388675746579118442610737887429869253658299129509604208337673171179422345943853427201429004780604235372282190097683100404473534090577374840759302746534877460743441569930414948308203378961538708556404061945229638790248297802018538766395038815263085180710993474433681783381461233039780119324361700914363644034748551122728658174102000289723679350260176920304357332226438091532173885076508215945639987504826028121418316410158147999776624241774872016480464050348034156135431544026190188755262061695410344766637407

In [26]:
#Vérifie la signature 
if(int(verif_signature(messageA[0],messageA[1],cléPrivéeCA,cléPubliqueA))==cléPubliqueA[0]):
    CAClePubliqeA=chiffrer(cléPubliqueA[0],cléPrivéeCA[0],cléPrivéeCA[1])
    print("Certificat de Alice: "+str(CAClePubliqeA))
else:
    print("Le message a été modifié")

Message déchiffré: 566104928636221625278900332184719948257355358737421014408807633408232712030899662110911989661372601715244421511622787813422758379783777015359452104631170063149460020877785979563563224060047210633852556439558230126894361217432020042783425309963720384665186488560701892836993628502147867457256010992570482653124199614636317912814797550716498362959083886757465791184426107378874298692536582991295096042083376731711794223459438534272014290047806042353722821900976831004044735340905773748407593027465348774607434415699304149483082033789615387085564040619452296387902482978020185387663950388152630851807109934744336817833814612330397801193243617009143636440347485511227286581741020002897236793502601769203043573322264380915321738850765082159456399875048260281214183164101581479997766242417748720164804640503480341561354315440261901887552620616954103447666374071348007015109544338980828501444149501283476119711424823510281335362179898735103113689947563614626105027861321487620872737434778

In [27]:
#Bob vérifie le certificat d'alice
print("Vérification clé publique Alice:")
verifCA=str(dechiffrer(CAClePubliqeA,cléPubliqueCA[0],cléPubliqueCA[1]))
if(verifCA==cléPubliqueA[0]):
    print(""+str(dechiffrer(CAClePubliqeA,cléPubliqueCA[0],cléPubliqueCA[1])))
else:
    print("Le message a été modifié")

Vérification clé publique Alice:
Le message a été modifié


In [28]:
#BOB
gB=generateurClé(10**19,10**20)
cléPubliqueB= [gB[0],gB[2]]
cléPrivéeB= [gB[1],gB[2]]
print("Clé Publique B: "+str(cléPubliqueB))
print("Clé Privée B: "+str(cléPrivéeB))

messagez=1435263726352725355365241625326643625165635625635624365265
messageCd=envoieAuthentifie(messagez,cléPrivéeB,cléPubliqueA)

print("Le message d'origine est "+str(messagez))

messageDk=verif_signature(messageCd[0],messageCd[1],cléPrivéeA,cléPubliqueB)

print("Le message décrypté est "+str(messageDk))

Clé Publique B: [30080808018615061709, 4824175432006288106675583108947522706673]
Clé Privée B: [494727510917771455347233198810248966053, 4824175432006288106675583108947522706673]
Empreinte: 2891672339950343496587536389153040848971999713978496074046960728399736954033327019602276680098853194260063673058514087405966492977229173518794508417971553
Empreinte Chiffré: 443127172329878667652072633906509975769647102457894200124669635746989924936183113245037616996886374470547464367861437204329971014673911541228640779545635031281930648146689212624578240909515321384326181632217425896751513577613683047098078755151141453118534971957205475476119640465216428314763605676975911005012074295430403244111210098027697861237881908856783420215557785034691209776834221302804852729714130327579977197534897400422302159605021791124787159692765073594445420515462940168346911656425742228622471353306430009324891123749321580439605571672004853563217726172447569246061065744686537372422412258913974383614810991278498058988318